In [1]:

import pandas as pd

In [33]:
import pytorch_lightning as pl
import torch
from torch.utils.data import DataLoader
from urb3d.segmentation.dataset import PointCloudSegmentationDataset, PointSampler, ChunkedPointCloudDataset
from urb3d.segmentation.segmentor import PointNetSegmentor

from sklearn.metrics import precision_recall_fscore_support

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [37]:
weighted_model = '../models/pointnet_weighted/best_results/epoch=31-val_loss=1.99-train_loss=1.35.ckpt'
unweighted_model = '../models/new_deal/best_results/epoch=89-val_loss=1.81-train_loss=1.32.ckpt'
# block_6_model = '../models/longrun_pointnet_weighted_ins/best_results/epoch=1-val_loss=2.42-train_loss=2.51.ckpt'
block_7_model = '../models/pointnet_weighted_isns/best_results/epoch=113-val_loss=0.79-train_loss=1.18.ckpt'
block_6_model = ('../models/pointnet_weighted_isns_6/best_results/epoch=45-val_loss=1.22-train_loss=1.41.ckpt')
block_6_7_model = ('../models/pointnet_weighted_isns_6_7/best_results/epoch=64-val_loss=2.01-train_loss=1.34.ckpt')

In [38]:
model = PointNetSegmentor.load_from_checkpoint(block_6_7_model, strict=False)
dataset = PointCloudSegmentationDataset('../data/birmingham_blocks/birmingham_block_7_subsampled_train.ply')
data_loader = DataLoader(dataset)
trainer = pl.Trainer()
preds = trainer.predict(model, data_loader)
preds = preds[0][0].cpu().numpy()
labels = data_loader.__iter__().__next__()[1][0].cpu().numpy()

C:\Users\kasia\PycharmProjects\urban-3d-reconstruction\venv\lib\site-packages\pytorch_lightning\core\saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['criterion.weight']
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\kasia\PycharmProjects\urban-3d-reconstruction\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

In [39]:
metrics = precision_recall_fscore_support(labels, preds)
for class_id in range(len(metrics[0])):
    for metric in range(4):
        print(round(metrics[metric][class_id],2), end='\t\t')
    print()
print(precision_recall_fscore_support(labels, preds, average='weighted'))

0.64		0.52		0.57		217554		
0.15		0.45		0.22		21855		
0.83		0.62		0.71		197866		
0.07		0.14		0.09		23066		
0.0		0.0		0.0		21730		
0.31		0.21		0.25		76932		
0.0		0.0		0.0		10098		
0.04		0.15		0.06		19145		
0.0		0.0		0.0		0		
0.0		0.0		0.0		0		
(0.5654087674084923, 0.4566745885224889, 0.49809829685214746, None)


C:\Users\kasia\PycharmProjects\urban-3d-reconstruction\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\kasia\PycharmProjects\urban-3d-reconstruction\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [30]:
for class_id in range(len(metrics[0])):
    for metric in range(4):
        print(round(metrics[metric][class_id],2), end='\t\t')
    print()

print(precision_recall_fscore_support(labels, preds, average='weighted'))

0.93		0.69		0.79		217554		
0.3		0.62		0.41		21855		
0.96		0.68		0.79		197866		
0.25		0.48		0.33		23066		
0.71		0.99		0.83		21730		
0.8		0.73		0.77		76932		
0.09		0.64		0.17		10098		
0.37		0.56		0.44		19145		
(0.5068797006883353, 0.44646997279765643, 0.4557179141521637, None)


In [ ]:
2

In [19]:
labels.shape

torch.Size([1, 238950])

In [11]:
preds[0][0].shape

torch.Size([238950])

In [9]:
for pred in preds:
    print(pred.shape)

torch.Size([1, 238950])


In [9]:
len(data_loader.dataset.pt.points)

238950

In [7]:
data_loader.dataset.point_sampler.subsample_size

In [15]:
print(data_loader.dataset.subsample_size)

238950


In [6]:
data_loader.__iter__().__next__()[0].shape

torch.Size([1, 3])

In [13]:
preds[0].shape

torch.Size([1, 1024])